## Welcome

The **Py**thon package **f**or **A**coustics **R**esearch (pyfar) contains classes and functions for the acquisition, inspection, and processing of audio signals. This is the pyfar demo notebook and a good place for getting started. In this notebook, you will see examples of the most important pyfar functionalty. 

**Note:** This is not a substitute for the [pyfar documentation](https://pyfar.readthedocs.io/en/latest/) that provides a complete description of the pyfar functionality.


## Getting started

Please note that this is not a Python tutorial. We assume that you are aware of basic Python coding and concepts including the use of `conda` and `pip`. If you did not install pyfar already please do so by running the command

`pip install pyfar`

After this go to your Python editor of choice and import pyfar

In [ ]:
# import packages
import pyfar as pf
import numpy as np

## Handling audio data

Audio data are the basis of pyfar and there are three classes for storing and handling it. Most data will be stored in objects of the `Signal` class, which is intended for time and frequency data that was sampled at equidistant times and frequencies. Examples for this are audio recordings or single sided spectra between 0 Hz and half the sampling rate.

The other two classes `TimeData` and `FrequencyData` are intended to store inclomplete audio data. For example time signals that were not sampled at equidistant times or frequency data that are not available for all frequencies between 0 Hz and half the sampling rate. We will only look at `Signals`, however, `TimeData`and `FrequencyData` are very similar.

`Signals` are stored along with information about the sampling rate, the domain (`time`, or `freq`), the FFT normalization and an optional comment. Lets go ahead and create a single channel signal

In [ ]:
# create a dirac signal with a sampling rate of 44.1 kHz
fs = 44100
x = np.zeros(44100)
x[0] = 1
x_energy = pf.Signal(x, fs)

# show information
x_energy

### FFT Normalization

Different FFT normalization are available that scale the spectrum of a Signal. Pyfar knows six normalizations: `'amplitude'`, `'rms'`, `'power'`, and `'psd'` from [Ahrens, et al. 2020](http://www.aes.org/e-lib/browse.cfm?elib=20838), `'unitary'` (only applies weights for single sided spectra as in Eq. 8 in [Ahrens, et al. 2020](http://www.aes.org/e-lib/browse.cfm?elib=20838)), and `'none'` (applies no normalization). The default normalization is `'none'`, which is useful for **energy signals**, i.e., signals with finite energy such as impulse responses. The other FFT normalizations are intended for **power signals**, i.e., samples of signals with infinite energy, such as noise or sine signals. Visit the [FFT concepts](https://pyfar.readthedocs.io/en/latest/concepts/pyfar.fft.html) for more information. Let's create a signal with a different normalization

In [ ]:
x = np.sin(2 * np.pi * 1000 * np.arange(441) / fs)
x_power = pf.Signal(x, fs, fft_norm='rms')
x_power

The normalization can be changed. In this case the spectral data of the signal is converted internally using `pyfar.fft.normalization()`

In [ ]:
x_power.fft_norm = 'amplitude'
x_power

Note the following: 

- The normalizations are only relevant for inspecting the magnitude data, i.e., in plotting. pyfar thus uses the non normalized spectra for all signal processing and arithmetic operations.
- `FrequencyData` objects do not support the FFT normalization because this requires knowledge about the sampling rate or the number of samples of the time signal. These objects thus have the `'none'` in all cases.
- `TimeData` does not support FFT normalization because it only exists in the time domain. These objects do not have the `fft_norm` attribute at all.

### Energy and power signals

You might have realized that pyfar distinguishes between energy and power signals, which is required for some operations. Signals with the FFT normalization `'none'` are considered as energy signals while all other FFT normalizations result in power signals.

### Accessing Signal data

You can access the data, i.e., the audio signal, inside a Signal object in the time and frequency domain by simply using

In [ ]:
time_data = x_power.time
freq_data_normalized = x_power.freq
freq_data_raw = x_power.freq_raw

Two things are important here:

1. The data are mutable! That means `x_power.time` changes if you change `time_data`. If this is not what you want use `time_data = x_power.time.copy()` instead.

2. The frequency data of signals is available without normalization from `x_power.freq_raw` and with normalization from `x_power.freq`. The latter depends on the Signal's `fft_norm`.

`Signals` and some other pyfar objects support slicing. Let's illustrate that for a two channel signal

In [ ]:
# generate two channel time data
time = np.zeros((2, 4))
time[0,0] = 1   # first sample of first channel
time[1,0] = 2   # first sample of second channel

x_two_channels = pf.Signal(time, 44100)
x_first_channel = x_two_channels[0]

`x_first_channel` is a `Signal` object itself, which contains the first channel of `x_two_channels`:

In [ ]:
x_first_channel.time

A third option to access `Signals` is to copy it

In [ ]:
x_copy = x_two_channels.copy()

It is important to note that this returns an independent copy of `x_two_channels`. Note that `x_copy = x_two_channels` might not be wanted. In this case changes to `x_copy` will also change `x_two_channels`. The `copy()` operation is available for all pyfar objects.

### Iterating Signals

It is the aim of pyfar that all operations work on N-dimensional `signals`. Nevertheless, you can also iterate `signals` if you need to apply operations depending on the channel. Lets look at a simple example

In [ ]:
signal = pf.Signal([[0, 0, 0], [1, 1, 1]], 44100)  # 2-channel signal

# iterate the signal
for n, channel in enumerate(signal):
    print(f"Channel: {n}, time data: {channel.time}")
    # do something channel dependent
    channel.time = channel.time + n
    # write changes to the signal
    signal[n] = channel

# q.e.d.
print(f"\nNew signal time data:\n{signal.time}")

`Signal` uses the standard `numpy` iterator which always iterates the first dimension. In case of a 2-D array as in the example above these are the channels.

### Signal meta data

The `Signal` object also holds useful metadata. The most important might be:

- `Signal.n_samples`: The number of samples in each channel (`Signal.time.shape[-1]`)
- `Signal.n_bins`: The number of frequencies in each channel (`Signal.freq.shape[-1]`)
- `Signal.times`: The sampling times of `Signal.time` in seconds
- `Signal.frequencies`: The frequencies of `Signal.freq` in Hz
- `Signal.comment`: A comment for documenting the signal content


### Arithmetic operations

The arithmetic operations `add`, `subtract`, `multiply`, `divide`, and `power` are available for `Signal` (time or frequency domain operations) as well as for `TimeData` and `FrequencyData`. The operations work on arbitrary numbers of Signals and array likes. Lets check out simple examples

In [ ]:
# add two signals energy signals
x_sum = pf.add((x_energy, x_energy), 'time')
x_sum.time

In this case, `x_sum` is also an energy Signal. However, this can be different for other operation as described in the documentation ([arithmetic operations](https://pyfar.readthedocs.io/en/latest/concepts/pyfar.arithmetic_operations.html)). Under the hood, the operations use numpy's [array broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html?highlight=broadcast#module-numpy.doc.broadcasting). This means you can add scalars, vectors, and matrixes to a signal. Lets have a frequency domain example for this

In [ ]:
x_sum = pf.add((x_energy, 1), 'freq')
x_sum.time

The Python operators `+`, `-`, `*`, `/`, `**` and `@` are overloaded with the **frequency domain** arithmetic functions for `Signal` and `FrequencyData`. For `TimeData` they correspond to time domain operations. Thus, the example above can also be shortened to

In [ ]:
x_sum = x_energy + 1
x_sum.time

### Plotting

Inspecting signals can be done with the `pyfar.plot` module, which uses common plot functions based on `Matplotlib`. For example a plot of the magnitude spectrum

In [ ]:
pf.plot.freq(x_power)

We set the FFT normalization to 'amplitude' before. The plot thus shows the amplitude (1, or 0 dB) of our sine wave contained in `x_power`. We can also plot the RMS ($1/\sqrt(2)$, or $\approx-3$ dB)

In [ ]:
x_power.fft_norm = 'rms'
pf.plot.line.freq(x_power)

### Interactive plots

pyfar provides keyboard shortcuts for switching plots, zooming in and out, moving along the x and y axis, and for zooming and moving the range of colormaps. To do this, you need to use an interactive Matplotlib backend. This can for example be done by including

`%matplotlib qt`

or

`matplotlib.use('Qt5Agg')`

in your code. These are the available keyboard short cuts

In [ ]:
shortcuts = pf.plot.shortcuts()

Note that additional controls are available through Matplotlib's [interactive navigation](https://matplotlib.org/3.1.1/users/navigation_toolbar.html).

### Manipulating plots

In many cases, the layout of the plot should be adjusted, which can be done using Matplotlib and the axes handle that is returned by all plot functions. For example, the range of the x-axis can be changed.

In [ ]:
ax = pf.plot.time(x_power)
ax.set_xlim(0, 2)

Note: For an easy use of the pyfar plotstyle (available as 'light' and 'dark' theme) wrappers for Matplotlib's `use` and `context` are available as `pyfar.plot.use` and `pyfar.plot.context`.

## Signals

The `pyfar.signals` module contains a variety of common audio signals including sine signals, sweeps, noise and pulsed noise. For brevity, lets have only one example


In [ ]:
sweep = pf.signals.exponential_sweep_time(2**12, [100, 22050])
pf.plot.time_freq(sweep)

## DSP

`pyfar.dsp` offers lots of useful functions to manipulate audio data including

- convolution, deconvolution, and regulated inversion
- windowing and zero-padding
- generating linear and minimum phase signals

and many more. Have a look at the [module documentation](https://pyfar.readthedocs.io/en/latest/modules/pyfar.dsp.html) for a complete overview.


### Filtering

`pyfar.dsp.filter` contains wrappers for the most common filters of `scipy.signal` as well as more audio specific filter such as shelve and bell filters. Visit the [filter types](https://pyfar.readthedocs.io/en/latest/concepts/pyfar.filter_types.html) for an overview.

All filters can used in a similar manner, like this one


In [ ]:
x_filter = pf.dsp.filter.bell(x_energy, center_frequency=1e3, gain=10, quality=2)
pf.plot.line.freq(x_filter)

## In'n'out

Now that you know what pyfar is about, let's see how you can save your work and read common data types.

### Read and write pyfar datan

Pyfar contains functions for saving all pyfar objects and common data types such as numpy arrays using `pf.io.write()` and `pf.io.read()`. This creates .far files that also support compression.

### Read and write audio-files

Audio-files in wav or other formats are commonly used in the audio community to store and exchange data. You can read them with

`signal = pf.io.read_audio(filename)`

and write them with

`pf.io.write_audio(signal, filename, overwrite=True)`.

You can write any `signal` to an audio, but in some cases, clipping will occur for values > 1. Multidimensional `signals` will be reshaped to 2D arrays before writing.

### Read SOFA files

[SOFA files](https://www.sofaconventions.org) can be used to store spatially distributed acoustical data sets. Examples for this are room acoustic measurements at different positions in a room or a set of head-related transfer functions for different source positions. SOFA files can quickly be read by

`signal, source, receiver =  pf.io.read_sofa(filename)`

which returns the audio data as a `Signal` or `FrequencyData` object (depending on the data in the SOFA file) and the source and receiver coordinates as a `Coordinates` object.

`read_sofa` uses the [sofar package](https://sofar.readthedocs.io/en/latest/), which we recommend to manipulate and write SOFA files or access the detailed meta data contained in SOFA files.